In [918]:
import yfinance as yf

def get_data(symbol: str):
    data = yf.download(tickers=symbol, period='50d', interval='1h')
    data.reset_index(inplace=True, drop=False)
    return data
# Get the data
data = get_data('BTC-USD')
data

[*********************100%%**********************]  1 of 1 completed


,Datetime,Open,High,Low,Close,Adj Close,Volume
0,2024-04-08 00:00:00+00:00,69360.914062,69414.273438,69124.523438,69309.898438,69309.898438,20531200
1,2024-04-08 01:00:00+00:00,69297.296875,69769.148438,69282.906250,69413.968750,69413.968750,0
2,2024-04-08 02:00:00+00:00,69423.085938,69423.085938,69161.109375,69196.078125,69196.078125,0
3,2024-04-08 03:00:00+00:00,69211.375000,69466.437500,69149.734375,69450.250000,69450.250000,24975360
4,2024-04-08 04:00:00+00:00,69455.554688,69477.109375,69294.539062,69411.382812,69411.382812,0
...,...,...,...,...,...,...,...
1187,2024-05-27 11:00:00+00:00,68535.640625,68605.812500,68234.132812,68401.351562,68401.351562,668553216
1188,2024-05-27 12:00:00+00:00,68392.507812,68586.226562,68353.484375,68586.226562,68586.226562,271726592
1189,2024-05-27 13:00:00+00:00,68570.500000,68977.117188,68570.500000,68883.007812,68883.007812,735604736
1190,2024-05-27 14:00:00+00:00,68893.882812,69420.296875,68832.203125,69406.625000,69406.625000,800071680


In [919]:
import pandas_ta as ta

def calculate_sma(data, length: int):
    return ta.sma(data['Close'], length)

# Calculate the moving average
data['SMA'] = calculate_sma(data, 20)
data.dropna(inplace=True)

## 1 - Slope of MAs

In [920]:
import numpy as np

def Get_Direction_By_SMA_Slope(series, period: int = 5):
    slopes = [0 for _ in range(period-1)]
    for i in range(period-1, len(series)):
        x = np.arange(period)
        y = series[i-period+1:i+1].values
        slope = np.polyfit(x, y, 1)[0]  # Calculate the slope using linear regression
        percent_slope = (slope / y[0]) * 100  # Convert the slope to a percentage
        slopes.append(percent_slope)
    return slopes

In [921]:
# Calculate the slope
data['Direction_By_SMA_Slope'] = Get_Direction_By_SMA_Slope(data['SMA'])
data.reset_index(inplace=True, drop=True)

In [922]:
#data[40:55]

In [923]:
import plotly.graph_objects as go

dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

fig.add_scatter(x=dfpl.index, y=dfpl['SMA'],  marker=dict(size=5, color="MediumPurple"), name="SMA")
fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [924]:
#data

## 2 - 3 MAs alignment

In [925]:
# Calculate the moving averages
data['SMA_10'] = calculate_sma(data, 10)
data['SMA_20'] = calculate_sma(data, 20)
data['SMA_30'] = calculate_sma(data, 30)

In [926]:
#data

In [927]:
def Get_Direction_By_SMAs_Trend(data):
    if data['SMA_10'] > data['SMA_20'] > data['SMA_30']:
        return 2  # Uptrend
    elif data['SMA_10'] < data['SMA_20'] < data['SMA_30']:
        return 1  # Downtrend
    else:
        return 0  # No trend

# Determine the trend and add it as a new column to the DataFrame
data['Direction_By_SMAs_Trend'] = data.apply(Get_Direction_By_SMAs_Trend, axis=1)

In [928]:
#data

In [929]:
import plotly.graph_objects as go

dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

# Add the moving averages to the plot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['SMA_10'], mode='lines', name='SMA 10', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['SMA_20'], mode='lines', name='SMA 20', line=dict(color='red')))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['SMA_30'], mode='lines', name='SMA 30', line=dict(color='green')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

## 3 - Candles above or below the MA curve

In [930]:
def Get_Direction_By_Candel_Position(data, backcandles, ref_column):
    categories = [0 for _ in range(backcandles)]
    for i in range(backcandles, len(data)):
        if all(data['Close'][i-backcandles:i] > data[ref_column][i-backcandles:i]):
            categories.append(2)  # Uptrend
        elif all(data['Close'][i-backcandles:i] < data[ref_column][i-backcandles:i]):
            categories.append(1)  # Downtrend
        else:
            categories.append(0)  # No trend
    return categories

# Apply the function to the DataFrame
data['Direction_By_Candel_Position_SMA20'] = Get_Direction_By_Candel_Position(data, 5, 'SMA_20')


In [931]:
#data[25:55]

In [932]:
import plotly.graph_objects as go

dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

# Add the moving averages to the plot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['SMA_20'], mode='lines', name='SMA 20', line=dict(color='red')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

## 4 - Apply trend detection using the VWAP curve

In [933]:
data

,Datetime,Open,High,Low,Close,Adj Close,Volume,SMA,Direction_By_SMA_Slope,SMA_10,SMA_20,SMA_30,Direction_By_SMAs_Trend,Direction_By_Candel_Position_SMA20
0,2024-04-08 19:00:00+00:00,71950.757812,71982.562500,71656.507812,71656.507812,71656.507812,88633344,71015.467188,0.000000,NaN,NaN,NaN,0,0
1,2024-04-08 20:00:00+00:00,71735.859375,71783.515625,71507.007812,71675.429688,71675.429688,431599616,71133.743750,0.000000,NaN,NaN,NaN,0,0
2,2024-04-08 21:00:00+00:00,71702.523438,71872.312500,71633.937500,71684.421875,71684.421875,47509504,71247.266406,0.000000,NaN,NaN,NaN,0,0
3,2024-04-08 22:00:00+00:00,71707.679688,71938.562500,71615.382812,71938.562500,71938.562500,53313536,71384.390625,0.000000,NaN,NaN,NaN,0,0
4,2024-04-08 23:00:00+00:00,71908.976562,71982.062500,71627.304688,71634.156250,71634.156250,0,71493.585938,0.169947,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1168,2024-05-27 11:00:00+00:00,68535.640625,68605.812500,68234.132812,68401.351562,68401.351562,668553216,68671.143359,-0.036841,68626.744531,68671.143359,68811.664844,1,1
1169,2024-05-27 12:00:00+00:00,68392.507812,68586.226562,68353.484375,68586.226562,68586.226562,271726592,68660.993359,-0.033843,68578.428125,68660.993359,68795.163281,1,1
1170,2024-05-27 13:00:00+00:00,68570.500000,68977.117188,68570.500000,68883.007812,68883.007812,735604736,68668.712891,-0.021008,68584.632812,68668.712891,68781.231771,1,1
1171,2024-05-27 14:00:00+00:00,68893.882812,69420.296875,68832.203125,69406.625000,69406.625000,800071680,68700.254687,0.002332,68647.323438,68700.254687,68790.610417,1,0


In [934]:
data.index =data.Datetime

data.ta.vwap(append=True)


C:\Users\alada\AppData\Local\Temp\ipykernel_40604\269704760.py:3: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



Datetime
2024-04-08 19:00:00+00:00    71765.192708
2024-04-08 20:00:00+00:00    71674.037377
2024-04-08 21:00:00+00:00    71678.739151
2024-04-08 22:00:00+00:00    71691.795651
2024-04-08 23:00:00+00:00    71691.795651
                                 ...     
2024-05-27 11:00:00+00:00    68675.985857
2024-05-27 12:00:00+00:00    68665.083276
2024-05-27 13:00:00+00:00    68686.842218
2024-05-27 14:00:00+00:00    68761.554362
2024-05-27 15:00:00+00:00    68775.430860
Name: VWAP_D, Length: 1173, dtype: float64

In [935]:
# Apply the check_candles function
data['Direction_By_Candel_Position_VWAP_D'] = Get_Direction_By_Candel_Position(data, 5, 'VWAP_D') 

In [936]:
data[data["Direction_By_Candel_Position_VWAP_D"]!=0]

,Datetime,Open,High,Low,Close,Adj Close,Volume,SMA,Direction_By_SMA_Slope,SMA_10,SMA_20,SMA_30,Direction_By_SMAs_Trend,Direction_By_Candel_Position_SMA20,VWAP_D,Direction_By_Candel_Position_VWAP_D
Datetime,,,,,,,,,,,,,,,,
2024-04-09 08:00:00+00:00,2024-04-09 08:00:00+00:00,70412.453125,70566.078125,69783.820312,70518.312500,70518.312500,0,71435.812109,-0.112417,71091.877344,NaN,NaN,0,0,71136.144090,1
2024-04-09 09:00:00+00:00,2024-04-09 09:00:00+00:00,70528.835938,70775.421875,70293.750000,70399.914062,70399.914062,0,71358.976562,-0.119736,70968.453125,NaN,NaN,0,0,71136.144090,1
2024-04-09 10:00:00+00:00,2024-04-09 10:00:00+00:00,70434.382812,70662.328125,70304.382812,70612.460938,70612.460938,0,71304.004687,-0.110077,70887.335156,NaN,NaN,0,0,71136.144090,1
2024-04-09 11:00:00+00:00,2024-04-09 11:00:00+00:00,70584.312500,70899.226562,70584.312500,70765.812500,70765.812500,0,71254.362109,-0.092698,70835.246094,NaN,NaN,0,0,71136.144090,1
2024-04-09 12:00:00+00:00,2024-04-09 12:00:00+00:00,70797.632812,70917.070312,70688.453125,70787.578125,70787.578125,0,71205.589844,-0.079100,70764.869531,NaN,NaN,0,0,71136.144090,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-27 09:00:00+00:00,2024-05-27 09:00:00+00:00,68589.625000,68597.554688,68435.898438,68571.554688,68571.554688,0,68725.884375,-0.037902,68713.471875,68725.884375,68849.164844,1,1,68740.784732,1
2024-05-27 10:00:00+00:00,2024-05-27 10:00:00+00:00,68561.070312,68561.070312,68409.859375,68539.085938,68539.085938,143255552,68691.030469,-0.036030,68693.593750,68691.030469,68835.113542,0,1,68730.254794,1
2024-05-27 11:00:00+00:00,2024-05-27 11:00:00+00:00,68535.640625,68605.812500,68234.132812,68401.351562,68401.351562,668553216,68671.143359,-0.036841,68626.744531,68671.143359,68811.664844,1,1,68675.985857,1


In [937]:
import plotly.graph_objects as go

dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

# Add the moving averages to the plot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['VWAP_D'], mode='lines', name='VWAP', line=dict(color='red')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [938]:
#data[:50]

## 5 - Trend confirmation using the ADX

In [939]:
# Calculate the ADX
data.ta.adx(append=True)

,ADX_14,DMP_14,DMN_14
Datetime,,,
2024-04-08 19:00:00+00:00,NaN,NaN,NaN
2024-04-08 20:00:00+00:00,NaN,NaN,NaN
2024-04-08 21:00:00+00:00,NaN,NaN,NaN
2024-04-08 22:00:00+00:00,NaN,NaN,NaN
2024-04-08 23:00:00+00:00,NaN,NaN,NaN
...,...,...,...
2024-05-27 11:00:00+00:00,15.297654,20.354359,28.882364
2024-05-27 12:00:00+00:00,15.442137,19.031947,27.005892
2024-05-27 13:00:00+00:00,14.805979,27.429958,24.064328


In [940]:
#data

In [941]:
# Define a function to generate the trend signal based on ADX
def Get_Direction_By_ADX(data, threshold=40):
    trend_signal = []
    for i in range(len(data)):
        if data['ADX'][i] > threshold:
            if data['DMP'][i] > data['DMN'][i]:
                trend_signal.append(2)  # Confirmed Uptrend
            else:
                trend_signal.append(1)  # Confirmed Downtrend
        else:
            trend_signal.append(0)  # No confirmed trend
    return trend_signal

In [942]:
# Apply the function to generate the trend signal column
data = data.rename(columns=lambda x: x[:-3] if x.startswith('ADX') else x)
data = data.rename(columns=lambda x: x[:-3] if x.startswith('DM') else x)

data['Direction_By_ADX'] = Get_Direction_By_ADX(data)

C:\Users\alada\AppData\Local\Temp\ipykernel_40604\54201453.py:5: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`

C:\Users\alada\AppData\Local\Temp\ipykernel_40604\54201453.py:6: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [943]:
data[data['Direction_By_ADX']!=0]

,Datetime,Open,High,Low,Close,Adj Close,Volume,SMA,Direction_By_SMA_Slope,SMA_10,SMA_20,SMA_30,Direction_By_SMAs_Trend,Direction_By_Candel_Position_SMA20,VWAP_D,Direction_By_Candel_Position_VWAP_D,ADX,DMP,DMN,Direction_By_ADX
Datetime,,,,,,,,,,,,,,,,,,,,
2024-04-09 22:00:00+00:00,2024-04-09 22:00:00+00:00,69185.125000,69394.734375,69152.976562,69320.789062,69320.789062,0,69954.210547,-0.161439,69143.551562,69954.210547,NaN,0,1,69898.048796,1,61.758572,12.133528,34.590303,1
2024-04-09 23:00:00+00:00,2024-04-09 23:00:00+00:00,69312.078125,69312.078125,68895.148438,69148.718750,69148.718750,93663232,69847.322656,-0.154417,69051.781250,69847.322656,NaN,0,1,69886.290155,1,60.808598,11.255137,36.471369,1
2024-04-10 00:00:00+00:00,2024-04-10 00:00:00+00:00,69137.804688,69271.750000,69034.648438,69118.390625,69118.390625,0,69744.091406,-0.151086,69042.072656,69744.091406,70338.069531,1,1,NaN,1,59.988479,10.786686,34.953389,1
2024-04-10 01:00:00+00:00,2024-04-10 01:00:00+00:00,69103.359375,69205.742188,68641.492188,69197.328125,69197.328125,0,69652.061328,-0.147221,69046.017187,69652.061328,70256.096875,1,1,NaN,0,59.932545,9.746990,38.300357,1
2024-04-10 02:00:00+00:00,2024-04-10 02:00:00+00:00,69214.687500,69568.554688,68540.406250,68719.851562,68719.851562,486952960,69554.750000,-0.142119,69055.845312,69554.750000,70157.577604,1,1,68942.937500,0,57.997848,13.809466,32.208405,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-24 11:00:00+00:00,2024-05-24 11:00:00+00:00,67324.750000,67413.187500,67272.773438,67320.554688,67320.554688,0,67539.164844,-0.066078,67393.738281,67539.164844,68064.963802,1,1,67344.541997,0,44.623522,11.585538,31.132072,1
2024-05-24 12:00:00+00:00,2024-05-24 12:00:00+00:00,67309.453125,67535.789062,67309.453125,67495.609375,67495.609375,0,67523.588672,-0.039470,67348.660156,67523.588672,67995.938281,1,1,67344.541997,0,44.185887,13.276187,29.896037,1
2024-05-24 13:00:00+00:00,2024-05-24 13:00:00+00:00,67470.500000,67539.320312,66923.351562,67065.156250,67065.156250,0,67495.660937,-0.036341,67272.064062,67495.660937,67907.058854,1,1,67344.541997,0,44.414443,11.892362,33.313435,1


In [944]:
# Direction_By_SMAs_Trend
# Get_Direction_By_ADX
data['Confirmed Signal'] = data.apply(lambda row: row['Direction_By_Candel_Position_VWAP_D'] if row['Direction_By_Candel_Position_VWAP_D'] == row['Direction_By_ADX'] else 0, axis=1)

In [945]:
#data[ (data["Direction_By_Candel_Position_VWAP_D"] != data["Direction_By_ADX"]) &   data['Confirmed Signal']!=0  ]
data[data['Confirmed Signal']!=0]

,Datetime,Open,High,Low,Close,Adj Close,Volume,SMA,Direction_By_SMA_Slope,SMA_10,...,SMA_30,Direction_By_SMAs_Trend,Direction_By_Candel_Position_SMA20,VWAP_D,Direction_By_Candel_Position_VWAP_D,ADX,DMP,DMN,Direction_By_ADX,Confirmed Signal
Datetime,,,,,,,,,,,,,,,,,,,,,
2024-04-09 22:00:00+00:00,2024-04-09 22:00:00+00:00,69185.125000,69394.734375,69152.976562,69320.789062,69320.789062,0,69954.210547,-0.161439,69143.551562,...,NaN,0,1,69898.048796,1,61.758572,12.133528,34.590303,1,1
2024-04-09 23:00:00+00:00,2024-04-09 23:00:00+00:00,69312.078125,69312.078125,68895.148438,69148.718750,69148.718750,93663232,69847.322656,-0.154417,69051.781250,...,NaN,0,1,69886.290155,1,60.808598,11.255137,36.471369,1,1
2024-04-10 00:00:00+00:00,2024-04-10 00:00:00+00:00,69137.804688,69271.750000,69034.648438,69118.390625,69118.390625,0,69744.091406,-0.151086,69042.072656,...,70338.069531,1,1,NaN,1,59.988479,10.786686,34.953389,1,1
2024-04-12 23:00:00+00:00,2024-04-12 23:00:00+00:00,67066.062500,67246.179688,66847.109375,67205.617188,67205.617188,497758208,69248.480859,-0.250738,67779.428125,...,69622.743490,1,1,67653.381033,1,41.542616,8.648417,41.931003,1,1
2024-04-13 00:00:00+00:00,2024-04-13 00:00:00+00:00,67147.718750,67147.718750,66804.320312,66902.976562,66902.976562,355815424,69037.275391,-0.267703,67537.343750,...,69516.254687,1,1,66951.671875,1,43.302229,8.257051,40.516024,1,1
2024-04-13 01:00:00+00:00,2024-04-13 01:00:00+00:00,66923.164062,66923.164062,65809.250000,65809.250000,65809.250000,1360953344,68779.078906,-0.298644,67180.778906,...,69364.815625,1,1,66340.375506,1,45.416688,7.273175,46.332611,1,1
2024-04-13 02:00:00+00:00,2024-04-13 02:00:00+00:00,65898.859375,66557.476562,65898.859375,66500.890625,66500.890625,887877632,68558.965234,-0.320460,66941.228125,...,69231.181771,1,1,66333.114720,1,47.379716,6.696013,42.655891,1,1
2024-04-30 19:00:00+00:00,2024-04-30 19:00:00+00:00,60319.816406,60391.652344,59120.066406,59120.066406,59120.066406,1606248448,61978.702539,-0.271516,60691.809766,...,62356.176302,1,1,61374.296829,1,41.304512,6.478162,40.381062,1,1
2024-04-30 20:00:00+00:00,2024-04-30 20:00:00+00:00,59163.242188,60052.585938,59163.242188,59848.996094,59848.996094,1998991360,61743.094531,-0.321323,60504.049609,...,62256.218359,1,1,61137.325160,1,43.522085,5.761789,35.915613,1,1


In [946]:
import plotly.graph_objects as go

dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close'])])

# Add the moving averages to the plot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['VWAP_D'], mode='lines', name='VWAP', line=dict(color='red')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()